In [1]:
import os
import numpy as np
from PIL import Image
from src.data_handling.mmg_detection_datasets import *
from src.visualizations.plot_image import plot_image_opencv_fit_window
from src.data_augmentation.breast_density.data.resize_image import *
from src.models import nets
from src.data_loader import ALLDataset
import torch
from torch.utils.data import DataLoader

from src.preprocessing.histogram_standardization import get_hist_stand_landmarks, apply_hist_stand_landmarks

# Cropped scans GPU Server
# info_csv='/home/lidia-garrucho/datasets/OPTIMAM/png_screening_cropped_fixed/client_images_screening.csv'
# dataset_path='/home/lidia-garrucho/datasets/OPTIMAM/png_screening_cropped_fixed/images'
inbreast_path="/home/lidia-garrucho/datasets/INBREAST/AllPNG_cropped"
inbreast_csv="/home/lidia-garrucho/datasets/INBREAST/INbreast_updated_cropped_breast.csv"
inbreast_loader = DataLoader(ALLDataset(inbreast_path, inbreast_csv, mode='train', data_loader_type='inbreast', load_max=20, batch_size=10))

KeyError: 'max_per_label'

In [ ]:
sample = next(iter(inbreast_loader))

In [ ]:
images, labels = sample

In [2]:
nets.AlexNetClassifier

src.models.nets.AlexNetClassifier

In [ ]:
def import_class(name):
    module_name, class_name = name.rsplit('.', 1)
    module = importlib.import_module(module_name)
    return getattr(module, class_name)


In [3]:
from pathlib import Path
import yaml
import importlib
config_file = Path('config.yaml')
with open(config_file) as file:
  CONFIG = yaml.safe_load(file)

In [4]:
AlexN = nets.AlexNetClassifier(in_ch=1, out_ch=1, pretrained=False)

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
ResNet18 = nets.ResNet18Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in ResNet18.parameters())
total_params

Freezing layers


Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.6.0


11177025

In [6]:
EfficientNetB0 = nets.EfficientNetB0Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in EfficientNetB0.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"


4008829

In [7]:
DenseNet161 = nets.DenseNet161Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in DenseNet161.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


26474209

In [8]:
ResNet50 = nets.ResNet50Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in ResNet50.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


23510081

In [9]:
EfficientNetB4 = nets.EfficientNetB4Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in EfficientNetB4.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


17550409

In [10]:
DenseNet121 = nets.DenseNet121Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in DenseNet121.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


6954881

In [11]:
DenseNet161 = nets.DenseNet161Classifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in DenseNet161.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


26474209

In [12]:
MobileNet = nets.MobileNetClassifier(in_ch=1, out_ch=1, pretrained=False)
total_params = sum(param.numel() for param in MobileNet.parameters())
total_params

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


2225153

In [17]:
import yaml
from pathlib import Path

import numpy as np
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
class AlexNetClassifier(nn.Sequential):
    def __init__(self, pretrained, in_ch, out_ch, seed=None, early_layers_learning_rate=0):
        '''
        in_ch = 1 or 3
        early_layers can be 'freeze' or 'lower_lr'
        '''
        super(AlexNetClassifier, self).__init__()
        torch.hub._validate_not_a_forked_repo=lambda a,b,c: True # no idea why it's needed, but it supposedly avoids the error "urllib.error.httperror http error 403 rate limit exceeded" in some centers
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=pretrained)
        # model.classifier[1]=nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1)) # Apply glorot initialization
        # self.relu = nn.ReLU()
        # self.fc = nn.Linear(1000, out_ch) # should adjust this
        self.model.classifier[6] = nn.Linear(4096, 1)
        

        if isinstance(self.model.classifier[6], nn.Linear):
            torch.nn.init.xavier_uniform_(self.model.classifier[6].weight)
            if self.model.classifier[6].bias is not None:
                torch.nn.init.zeros_(self.model.classifier[6].bias)

        # if out_ch == 1:
        #     self.out = nn.Sigmoid()
        # else:
        self.out = nn.Softmax(dim=1)
        super(AlexNetClassifier, self).__init__(self.model,
                                                # self.relu,
                                                # self.fc,
                                                 self.out)

In [113]:
AlexNC = AlexNetClassifier(in_ch=1, out_ch=1, pretrained=False)

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


In [104]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
model.classifier[6]

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


Linear(in_features=4096, out_features=1000, bias=True)

Linear(in_features=4096, out_features=1000, bias=True)

In [114]:
AlexNC(images).shape

torch.Size([1, 1])

In [35]:
class VGG19Classifier(nn.Sequential):
    def __init__(self, pretrained, in_ch, out_ch, seed=None, early_layers_learning_rate=0):
        '''
        in_ch = 1 or 3
        early_layers can be 'freeze' or 'lower_lr'
        '''
        super(VGG19Classifier, self).__init__()
        torch.hub._validate_not_a_forked_repo=lambda a,b,c: True # no idea why it's needed, but it supposedly avoids the error "urllib.error.httperror http error 403 rate limit exceeded" in some centers
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=pretrained)
        # model.classifier[1]=nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1)) # Apply glorot initialization
        self.model.classifier[6] = nn.Linear(4096, out_ch) # should adjust this

        if isinstance(self.model.classifier[6], nn.Linear):
            torch.nn.init.xavier_uniform_(self.model.classifier[6].weight)
            if self.model.classifier[6].bias is not None:
                torch.nn.init.zeros_(self.model.classifier[6].bias)

        if out_ch == 1:
            self.out = nn.Sigmoid()
        else:
            self.out = nn.Softmax(dim=1)
        super(VGG19Classifier, self).__init__(self.model, 
                                                 self.out)

In [36]:
VGG19 = VGG19Classifier(in_ch=1, out_ch=1, pretrained=False)

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


In [37]:
total_params = sum(param.numel() for param in VGG19.parameters())
total_params

139574337

In [38]:
VGG19

VGG19Classifier(
  (0): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (1

In [115]:
class DenseNet121Classifier(nn.Sequential):
    def __init__(self, pretrained, in_ch, out_ch, seed=None, early_layers_learning_rate=0):
        '''
        in_ch = 1 or 3
        early_layers can be 'freeze' or 'lower_lr'
        '''
        super(DenseNet121Classifier, self).__init__()
        torch.hub._validate_not_a_forked_repo=lambda a,b,c: True # no idea why it's needed, but it supposedly avoids the error "urllib.error.httperror http error 403 rate limit exceeded" in some centers
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=pretrained)
        # model.classifier[1]=nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1)) # Apply glorot initialization
        self.model.fc = nn.Linear(2048, out_ch) # should adjust this

        if isinstance(self.model.fc, nn.Linear):
            torch.nn.init.xavier_uniform_(self.model.fc.weight)
            if self.model.fc.bias is not None:
                torch.nn.init.zeros_(self.model.fc.bias)

        if out_ch == 1:
            self.out = nn.Sigmoid()
        else:
            self.out = nn.Softmax(dim=1)
        super(ResNet101Classifier, self).__init__(self.model, 
                                                 self.out)

In [116]:
pretrained=False
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=pretrained)

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


In [118]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=pretrained)

Using cache found in /home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/home/akis-linardos/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"


In [119]:
model

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      